# Multinominal Naive Bayes

In [1]:
from spacy.en import English
parser = English()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
import string
import re
import pandas as pd
import time
import os
import numpy as np
import pickle

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))

In [3]:
# DEFINE DIRECTORY PATH
path_to_json = './TrainingData2014/'

# CREATE LIST OF FILES FROM THE DIRECTORY
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# DEFINE PANDAS DATAFRAME
df = pd.DataFrame()

# LOOP THROUGH FILES, READ IN JSON AND BUILD DATAFRAME
for index, js in enumerate(json_files):
    json_data = pd.read_json(os.path.join(path_to_json, js), 'r')
    df = df.append(json_data)
    
# LOOK AT TOPIC DICTIONARY AND GET A TOPIC COUNT
topic_file = open('./topicDictionary.txt', 'r')
topics = topic_file.read().split('\r\n')

# SPLITTING THE ELEMENTS OF THE JSON INTO TEXT, PUBLICATION DATE AND TOPICS
df['text'] = df.TrainingData.apply(lambda x: x['bodyText'])
df['pubdate'] = df.TrainingData.apply(lambda x: x['webPublicationDate'])
df['topics'] = df.TrainingData.apply(lambda x: x['topics'])

# DROP FIRST TWO COLUMNS
df.reset_index(inplace=True, drop=True)
df.drop('TrainingData', axis=1, inplace=True)

# DEFINE FUNCTION TO CREATE OUR DATAFRAME
def topic_col(x):
    a = 0
    for elem in x:
        if elem == topic:
            a = 1
    return a

# RUN TOPIC COL FUNCTION ON ALL DATA
for topic in topics:
    time1 = time.time()
    df[topic] = df['topics'].map(topic_col)
    print topic
    time2 = time.time()
    time_in_s = (time2-time1)
    print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Function takes around 0.344 seconds to run
afghanistan
Function takes around 0.403 seconds to run
aid
Function takes around 0.346 seconds to run
algerianhostagecrisis
Function takes around 0.500 seconds to run
alqaida
Function takes around 0.465 seconds to run
alshabaab
Function takes around 0.453 seconds to run
antiwar
Function takes around 0.414 seconds to run
arabandmiddleeastprotests
Function takes around 0.338 seconds to run
armstrade
Function takes around 0.354 seconds to run
australianguncontrol
Function takes around 0.264 seconds to run
australiansecurityandcounterterrorism
Function takes around 0.273 seconds to run
bastilledaytruckattack
Function takes around 0.252 seconds to run
belgium
Function takes around 0.235 seconds to run
berlinchristmasmarketattack
Function takes around 0.291 seconds to run
bigdata
Function takes around 0.272 seconds to run
biometrics
Function takes around 0.246 seconds to run
bokoharam
Function takes around 0.237 seconds to run
bostonmaratho

undercoverpoliceandpolicing
Function takes around 0.270 seconds to run
unitednations
Function takes around 0.246 seconds to run
usguncontrol
Function takes around 0.281 seconds to run
values
Function takes around 0.270 seconds to run
warcrimes
Function takes around 0.275 seconds to run
warreporting
Function takes around 0.286 seconds to run
weaponstechnology
Function takes around 0.274 seconds to run
womeninbusiness
Function takes around 0.279 seconds to run
woolwichattack
Function takes around 0.277 seconds to run
worldmigration
Function takes around 0.285 seconds to run
zikavirus
Function takes around 0.281 seconds to run


In [4]:
def sampler(topic, df):
    
    # SELECT INDICES OF TOPIC ARTICLES
    topicindexes = df[df[topic] == 1].index.tolist()
    
    # FIND COUNT OF TOPIC ARTICLES
    articlecount = len(topicindexes)
    
    if articlecount < 10:
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*27, random_state=42).index.tolist()
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X = X.append(df.iloc[topicindexes]['text']).reset_index(drop=True)
        y = y.append(df.iloc[topicindexes][topic]).reset_index(drop=True)
        
    elif articlecount < 100:
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*9, random_state=42).index.tolist()
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text']).reset_index(drop=True)
        y = y.append(df.iloc[topicindexes][topic]).reset_index(drop=True)
        
    else:
        # SELECT NON TOPIC INDICES
        topicindexes = df[df[topic] == 1].sample(100).index.tolist()
        
        nontopicarticlesindexes = df[df[topic] == 0].sample(len(topicindexes)*9, random_state=42).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
    
    return X, y

In [5]:
# CREATE FINAL TEST DF
testdf = pd.read_json('TestData.json', 'r')
testdf['text'] = testdf.TestData.apply(lambda x: x['bodyText'])
testdf['pubdate'] = testdf.TestData.apply(lambda x: x['webPublicationDate'])
testdf['topics'] = testdf.TestData.apply(lambda x: x['topics'])
testdf.drop('TestData', axis=1, inplace=True)

In [6]:
submission = pd.DataFrame(index=testdf.index, columns=topics)

In [7]:
parameters = {
    'tfidf__norm': ('l2', 'l1'),
    'tfidf__use_idf': (True, False),
    'tfidf__sublinear_tf': (True, False), 
    'clf__penalty': ('l1', 'l2'),
    'clf__alpha': (1, 0.1, 0.001, 0.00001),
}

clf = MultinomialNB()

In [8]:
def printNMostInformative(vectorizer, clf, N):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Strongest Features: ")
    for feat in topClass2:
        print(feat)

In [9]:
# ADDED 4 THAT WERE NOT IN 2014 ARTICLES
topics_0 = ['activism', 'turkeycoupattempt', 'tunisiaattack2015', 'sanbernardinoshooting', 'peaceandreconciliation',
            'parisattacks', 'orlandoterrorattack', 'munichshooting', 'francetrainattack', 'charliehebdoattack',
            'brusselsattacks', 'berlinchristmasmarketattack','bastilledaytruckattack','zikavirus', 'jubilee', 'values', 'rio20earthsummit', 'algerianhostagecrisis']

In [12]:
time1 = time.time()

for topic in topics:
    if topic in topics_0:
        submission[topic] = 0
    else:
        X, y = sampler(topic, df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
        
        #N-GRAM RANGE OF (1,2)
        vectorizer = CountVectorizer(ngram_range=(1,4), max_df=0.2,
                                     stop_words=STOPLIST, strip_accents='unicode')

        # PIPELINE OF VECTORIZER, TF-IDF TRANSFORMER, STANDARD SCALER, MODEL
        pipe = Pipeline([('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()),
                 ('ss', StandardScaler(with_mean=False)),
                 ('clf', clf)])
        
        pipe.fit(X_train, y_train)

        # GRIDSEARCH PIPELINE FIT
        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # PREDICT
        preds = grid_search.best_estimator_.predict(X_test)
        topicf1score = f1_score(y_test, preds)
        print topic, "F1 score:", topicf1score
        print "Grid Search Best Params:", grid_search.best_estimator_.get_params
        
        print("----------------------------------------------------------------------------------------------")
        print("Top 10 features used to predict: ")
        printNMostInformative(vectorizer, clf, 30)
        
        # FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])

        # WRITE TO CSV
        submission[topic] = preds_final
        
        # PICKLE GRID SEARCH BEST PARAMETERS
        with open('Pickles/gs_logreg'+str(topic)+'.pickle', 'wb') as handle:
            pickle.dump(grid_search.best_estimator_, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    submission.to_csv('submission_full.csv')
    print topic
    
time2 = time.time()
time_in_s = (time2-time1)
print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  4.0min finished


afghanistan F1 score: 0.714285714286
Grid Search Best Params: <bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=se...th_mean=False, with_std=True)), ('clf', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))])>
----------------------------------------------------------------------------------------------
Top 10 features used to predict: 
Strongest Features: 
(-9.6802358912574391, u'afghanistan')
(-9.9404908450810723, u'afghan')
(-9.9823871885552755, u'taliban')
(-10.134651855832756, u'troops')
(-10.184150317410724, u'kabul')
(-10.199048239156578, u'hamid karzai')
(-10.202724874633901, u'military')
(-10.250025724468859, u'hamid')
(-10.267442665506646, u'abdullah abdullah')
(-10.26967452404

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  4.2min finished


aid F1 score: 0.88
Grid Search Best Params: <bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=se...with_mean=False, with_std=True)), ('clf', MultinomialNB(alpha=1, class_prior=None, fit_prior=True))])>
----------------------------------------------------------------------------------------------
Top 10 features used to predict: 
Strongest Features: 
(-9.8828764151445245, u'aid')
(-9.9580350543123384, u'development')
(-10.157160684432597, u'countries')
(-10.158587198248949, u'international')
(-10.202131192295365, u'international development')
(-10.234468081265927, u'department international development')
(-10.234468081265927, u'department international')
(-10.255173011560471, u'donors')
(-10.277514

...

womeninbusiness F1 score: 1.0
Grid Search Best Params: <bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=se...with_mean=False, with_std=True)), ('clf', MultinomialNB(alpha=1, class_prior=None, fit_prior=True))])>
----------------------------------------------------------------------------------------------
Top 10 features used to predict: 
Strongest Features: 
(-9.3949993504745102, u'female')
(-9.410930041311321, u'workplace')
(-9.4172524512982587, u'business')
(-9.461767390546699, u'successful')
(-9.4643656277015005, u'success')
(-9.527074738283714, u'models')
(-9.5325504945672819, u'women leadership')
(-9.5384943659412631, u'gender')
(-9.5519098644955278, u'industry')
(-9.552999946788967, u'beli

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  6.3min finished


woolwichattack F1 score: 0.869565217391
Grid Search Best Params: <bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=se...with_mean=False, with_std=True)), ('clf', MultinomialNB(alpha=1, class_prior=None, fit_prior=True))])>
----------------------------------------------------------------------------------------------
Top 10 features used to predict: 
Strongest Features: 
(-8.9623780650057654, u'lee rigby')
(-9.0122659136414178, u'rigby')
(-9.1217370418225272, u'michael adebowale')
(-9.1498212215842862, u'michael')
(-9.1566746971381079, u'michael adebolajo')
(-9.1883144799584215, u'adebowale')
(-9.2075537229511681, u'soldier')
(-9.2314348757331448, u'attack')
(-9.2466693386468499, u'terrorism')
(-9.

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  7.3min finished


worldmigration F1 score: 0.9375
Grid Search Best Params: <bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=se..._mean=False, with_std=True)), ('clf', MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True))])>
----------------------------------------------------------------------------------------------
Top 10 features used to predict: 
Strongest Features: 
(-9.7310440260018289, u'migrants')
(-9.7441437964597775, u'immigration')
(-9.8354056872918747, u'migration')
(-9.9223085957793664, u'eu')
(-9.930641442990229, u'david cameron')
(-9.9846846729018637, u'europe')
(-10.058203645651865, u'cameron')
(-10.087462127063137, u'benefits')
(-10.111243817244183, u'movement')
(-10.113002868422475, u'eur

Use our logistic regression from floydhub n(1,1) for (afghanistan?), big data, biometrics, bokoharam bostonmarathonbombing, cameroon, carers, (chemical weapons..hmm), cybercrime(hmm), 
dataprotection, drones(hmm), ebola, egypt, ethics, extradition(hmm), famine

To retrain - Clusterbombs, Australianguncontrol, European Arrest Warrant